In [31]:
import keras
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras import models
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Dropout, LSTM, Activation,SimpleRNN
%matplotlib inline
from decimal import Decimal
import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,Normalizer, MinMaxScaler
from sklearn.cross_validation import cross_val_score
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb

In [32]:
df=pd.read_csv('~/Downloads/NCSU-master-2/DC_ten_skus.csv')
#df=pd.read_csv('./DC_ten_skus.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.weekday_name
day_dummy=pd.get_dummies(df.Day)
df=pd.concat([df,day_dummy],axis=1)
df.drop(['Day','Date'],inplace=True,axis=1)

y=df['Sales']

In [33]:
df.head()
df.shape

(942, 16)

In [34]:
X=df.drop(['id','DayOfWeek','Customers','Sales'],inplace=False,axis=1)
X.head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,0,10,2013,1,0,0,0,0,0,1,0
1,10,0,10,2013,1,0,0,0,0,0,0,1
2,10,0,8,2013,1,0,0,0,0,1,0,0
3,10,0,8,2013,1,1,0,0,0,0,0,0
4,10,0,1,2013,1,0,0,1,0,0,0,0


In [35]:
for obs in range(1,8):
    X["Sales_T"+str(obs)]=df['Sales'].shift(obs)
X.head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Sales_T1,Sales_T2,Sales_T3,Sales_T4,Sales_T5,Sales_T6,Sales_T7
0,0,0,10,2013,1,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,0,10,2013,1,0,0,0,0,0,0,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,10,0,8,2013,1,0,0,0,0,1,0,0,60436.0,0.0,NaN,NaN,NaN,NaN,NaN
3,10,0,8,2013,1,1,0,0,0,0,0,0,52848.0,60436.0,0.0,NaN,NaN,NaN,NaN
4,10,0,1,2013,1,0,0,1,0,0,0,0,55606.0,52848.0,60436.0,0.0,NaN,NaN,NaN


In [36]:
X['Mov_avg']=df['Sales'].rolling( window=7).mean().shift(1) 
y=y[7:]
X=X[7:]
y.reset_index(drop=True, inplace=True)
X.reset_index(drop=True, inplace=True)
X.head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Sales_T1,Sales_T2,Sales_T3,Sales_T4,Sales_T5,Sales_T6,Sales_T7,Mov_avg
0,10,10,1,2013,1,0,0,0,0,0,1,0,89391.0,0.0,43727.0,55606.0,52848.0,60436.0,0.0,43144.000000
1,10,10,1,2013,1,0,0,0,0,0,0,1,72552.0,89391.0,0.0,43727.0,55606.0,52848.0,60436.0,53508.571429
2,10,10,1,2013,1,0,0,0,0,1,0,0,64297.0,72552.0,89391.0,0.0,43727.0,55606.0,52848.0,54060.142857
3,10,10,1,2013,1,1,0,0,0,0,0,0,64112.0,64297.0,72552.0,89391.0,0.0,43727.0,55606.0,55669.285714
4,10,0,0,2013,1,0,0,1,0,0,0,0,65006.0,64112.0,64297.0,72552.0,89391.0,0.0,43727.0,57012.142857


In [37]:
std = preprocessing.StandardScaler().fit(X)
X = std.transform(X)
len(X)

935

In [38]:
cv=5

RF=RandomForestRegressor( n_estimators=1000,max_depth=10,max_features=14);
rf_models=[];
rf_models=(cross_val_score(RF,X,y,cv=cv, scoring ='mean_absolute_error').mean().round(0) )


GB=ensemble.GradientBoostingRegressor(n_estimators= 1000,max_depth= 8, max_features=14,
          learning_rate= 0.01, loss= 'ls');
gb_models=[];
gb_models=(cross_val_score(GB,X,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )
   

XG=xgb.XGBRegressor(n_estimators= 900,learning_rate =0.01, max_depth=6,gamma=2)
xg_models=[]
xg_models=(cross_val_score(XG,X,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )

In [39]:
def create_MLP():
    mlp=models.Sequential()
    mlp.add(Dense(20, input_dim=X.shape[1], activation='relu'))
    #mlp.add(Dense(20, activation='relu'))
    mlp.add(Dense(30, activation='relu'))
    mlp.add(Dense(10, activation='relu'))    
    mlp.add(Dense(1, activation='relu'))
    mlp.compile(loss='mae', optimizer='adam', metrics=['mae'])
    return mlp

MLP = KerasRegressor(build_fn=create_MLP,
                               epochs=300,
                               batch_size=20,
                               verbose=0)

mlp_models=[]
mlp_models=(cross_val_score(MLP,X,y,cv=cv).mean().round(0)  )

In [40]:
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import ExpSineSquared
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process import GaussianProcessRegressor

kernels = [    
    1.0* RBF() + 
    1.0* RBF() *
    1.0* ExpSineSquared(periodicity=7.0,length_scale_bounds=(1,10),periodicity_bounds=(7, 7) )   
]

n_restarts_optimizer=10

GPR=GaussianProcessRegressor(kernel=kernels[0], n_restarts_optimizer=n_restarts_optimizer)  

gpr_models=[]
gpr_models=(cross_val_score(GPR,X,y,cv=cv, scoring='mean_absolute_error').mean().round(0))


In [41]:
len(X)
len(y)

935

In [42]:
#Summary
mae=-pd.DataFrame(np.column_stack((mlp_models,rf_models,gb_models,xg_models,gpr_models)) )
mae=mae.rename(columns = {0:'MLP',1:'RF',2:'GB',3:'XG',4:"GPR"})
best=mae.idxmin(axis=1)
best.astype('category')
mae["Best"]=best
mae

,MLP,RF,GB,XG,GPR,Best
0,3972.0,3182.0,3068.0,3118.0,52053.0,GB


In [43]:
#Same models as above, but configured here to predict over entire X (935 days)
def RF_final(X, y):
    tree=RF
    tree.fit(X,y)
    pred=tree.predict(X)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'RF'})
    return (pred, tree)

def MLP_final(X,y):
    model = MLP
    model.fit(X,y)
    pred=model.predict(X)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'MLP'})
    return (pred, model)

def GB_final(X,y):
    model = GB
    model.fit(X,y)
    prediction=model.predict(X)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'GB'})
    return (pred, model)

def XG_final(X,y):
    model=XG
    model.fit(X,y)
    prediction=model.predict(X)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'XG'})
    return (pred, model)

In [44]:
#Using aggregated SKU sales data, the best performing model is selected based on K-fold. 
#Then best model is used to predict full data set 

bm_dict={}
for sku in range(0,1):        
        if  mae["Best"][0]=="RF":
            bm_dict[sku]=RF_final(X,y);
                    
        elif mae["Best"][0]=="NN":
            bm_dict[sku]=MLP_final(X,y);
                                
        elif mae["Best"][0]=="GB":
            bm_dict[sku]=GB_final(X,y); 
                        
        elif mae["Best"][0]=="XG":
            bm_dict[sku]=XG_final(X,y);

In [45]:
bm_dict

{0: (               GB
  0    72391.268380
  1    64637.135388
  2    63662.901707
  3    65037.068279
  4    43222.875952
  5       -4.622564
  6    50906.169022
  7    48194.624996
  8    45987.568843
  9    47281.915524
  10   51605.783069
  11   42778.446471
  12      -1.739599
  13   71831.720976
  14   68099.165134
  15   65918.521704
  16   60980.788308
  17   63316.356103
  18   41831.236746
  19      -8.502152
  20   48784.100595
  21   47661.616857
  22   48679.803832
  23   58182.234020
  24   60483.836075
  25   48943.957240
  26      11.996494
  27   87957.495060
  28   74607.578126
  29   70671.607669
  ..            ...
  905  75528.345196
  906  71097.641953
  907  49622.294411
  908     -2.814498
  909  65275.436725
  910  57373.112902
  911  58335.469816
  912  58509.573678
  913  59797.805598
  914  46516.516943
  915     26.902992
  916  88761.308972
  917  79853.230864
  918  73343.745871
  919  72706.039554
  920  65064.381132
  921  48934.034690
  922     19.0964

In [46]:
# Create Phase 2 data set which is comprised of both aggregate SKU data and forecasts
dc_pred=pd.DataFrame(bm_dict[0][0])
# sku_pred=pd.read_csv('C:\\Users\\lengada1\\NCSU\\prediction_skus.csv')
sku_pred=pd.read_csv('./prediction_skus.csv')
sku_pred=sku_pred.iloc[:,1:]
X2=pd.concat([sku_pred,dc_pred,pd.DataFrame(X)],axis=1)
sku_pred
X2.head()
sku_pred,dc_pred.head()

(               1            2             3             4            5  \
 0    5693.193925  6303.152272   9845.807535  10206.146484  5585.067958   
 1    5539.595958  6555.860345   8411.639210   9118.957031  5824.338583   
 2    5066.336187  5485.939943   7731.928681   9120.459961  4943.748700   
 3    4861.325073  4855.833467   8811.058785   9824.867188  5083.340501   
 4    4894.417490  2579.798593   4403.234392   9802.511719  1814.601078   
 5       0.000000     0.000000      0.000000     -0.000415     0.000000   
 6    4481.734005  3982.376276   4959.057241   7667.046387  3900.943029   
 7    3991.253060  3931.732374   5114.052700   7145.960938  3744.720345   
 8    3985.849512  4502.407539   5034.825950   6966.017090  3882.577859   
 9    3940.036435  3621.608549   4793.865812   7111.237793  3568.472519   
 10   4257.835932  3732.825022   5648.779211   7862.683105  4142.704268   
 11   5071.419202  2442.788405   4043.308020   9452.623047  1740.935823   
 12      0.000000     0.0

In [47]:
X2.head()

,1,2,3,4,5,6,7,8,9,10,...,10,11,12,13,14,15,16,17,18,19
0,5693.193925,6303.152272,9845.807535,10206.146484,5585.067958,7147.661768,10084.369141,6433.098762,6374.213867,5688.395479,...,2.444916,-0.409012,1.370128,-1.931519,-0.315101,0.124641,0.024060,0.302992,-1.931749,-1.179223
1,5539.595958,6555.860345,8411.639210,9118.957031,5824.338583,6880.898734,9011.419922,5525.291577,5318.414062,5569.069536,...,-0.409012,2.444916,0.747023,1.370731,-1.930863,-0.314460,0.126098,0.021711,0.304175,0.161665
2,5066.336187,5485.939943,7731.928681,9120.459961,4943.748700,6295.468110,7932.394531,6176.105360,5331.561523,5589.729284,...,-0.409012,-0.409012,0.441558,0.748671,1.372236,-1.930804,-0.313389,0.123948,0.023445,0.233023
3,4861.325073,4855.833467,8811.058785,9824.867188,5083.340501,6896.488886,8253.934570,5165.642864,5739.909180,5609.136196,...,-0.409012,-0.409012,0.434712,0.443718,0.750016,1.373484,-1.931155,-0.316397,0.125482,0.441202
4,4894.417490,2579.798593,4403.234392,9802.511719,1814.601078,3787.978551,5127.479492,2563.439689,4959.999023,4493.968093,...,-0.409012,-0.409012,0.467794,0.436883,0.444984,0.751040,1.376040,-1.937323,-0.314000,0.614930


In [48]:
rng=range(0,33)
new_cols =  ['' + str(i) for i in rng]
X2.columns = new_cols[:32]
std2= preprocessing.StandardScaler().fit(X2)
X2 = std2.transform(X2)

In [49]:
pd.DataFrame(X2).head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.888719,0.938331,1.271612,0.599713,0.730175,1.041660,0.722144,0.684415,0.366587,0.466161,...,2.444916,-0.409012,1.370128,-1.931519,-0.315101,0.124641,0.024060,0.302992,-1.931749,-1.179223
1,0.810571,1.047609,0.826271,0.315812,0.832323,0.934110,0.441767,0.342357,-0.014689,0.413066,...,-0.409012,2.444916,0.747023,1.370731,-1.930863,-0.314460,0.126098,0.021711,0.304175,0.161665
2,0.569782,0.584944,0.615206,0.316204,0.456387,0.698085,0.159803,0.587581,-0.009942,0.422258,...,-0.409012,-0.409012,0.441558,0.748671,1.372236,-1.930804,-0.313389,0.123948,0.023445,0.233023
3,0.465475,0.312468,0.950300,0.500148,0.515981,0.940395,0.243826,0.206843,0.137523,0.430894,...,-0.409012,-0.409012,0.434712,0.443718,0.750016,1.373484,-1.931155,-0.316397,0.125482,0.441202
4,0.482312,-0.671757,-0.418427,0.494310,-0.879489,-0.312848,-0.573160,-0.773655,-0.144122,-0.065310,...,-0.409012,-0.409012,0.467794,0.436883,0.444984,0.751040,1.376040,-1.937323,-0.314000,0.614930


In [50]:
def create_MLP2():
    mlp=models.Sequential()
    mlp.add(Dense(40, input_dim=X2.shape[1], activation='relu'))
    #mlp.add(Dense(20, activation='relu'))
    mlp.add(Dense(30, activation='relu'))
    mlp.add(Dense(10, activation='relu'))    
    mlp.add(Dense(1, activation='relu'))
    mlp.compile(loss='mae', optimizer='adam', metrics=['mae'])
    return mlp

MLP2 = KerasRegressor(build_fn=create_MLP2,
                               epochs=600,
                               batch_size=20,
                               verbose=0)

In [51]:
rf2_models=[];
rf2_models=(cross_val_score(RF,X2,y,cv=cv, scoring ='mean_absolute_error').mean().round(0) )

gb2_models=[];
gb2_models=(cross_val_score(GB,X2,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )
   

xg2_models=[];
xg2_models=(cross_val_score(XG,X2,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )

In [52]:

mlp2_models=[];
mlp2_models=(cross_val_score(MLP2,X2,y,cv=cv, scoring='mean_absolute_error').mean().round(0)  )

In [53]:
mae2=-pd.DataFrame(np.column_stack((mlp2_models,rf2_models,gb2_models,xg2_models)) )
mae2=mae2.rename(columns = {0:'MLP',1:'RF',2:'GB',3:'XG'})
mae2

,MLP,RF,GB,XG
0,445.0,610.0,528.0,303.0


In [54]:
#Using forecasted data and no empirical sales data, the best performaning model is selected based on Train/Test. 
## NOTE this improvement is not the true delta.  Only a final forecast between Stage 1 vs 2 will tell. 

best2=mae2.idxmin(axis=1)
best2.astype('category')
mae2["Best"]=best2
both_mae=pd.concat([mae,mae2],axis=0)
both_mae.reset_index(drop=True, inplace=True)

# Phase 1 & 2 DC Models compared based on Test set MAE scores. 
both_mae['min']=both_mae.min(axis=1)
both_mae['error reduction']= ((both_mae['min'][0]-both_mae['min'][1])/both_mae['min'][0] ).round(3)
both_mae

,Best,GB,GPR,MLP,RF,XG,min,error reduction
0,GB,3068.0,52053.0,3972.0,3182.0,3118.0,3068.0,0.901
1,XG,528.0,NaN,445.0,610.0,303.0,303.0,0.901


In [55]:
bm2_dict={}
for sku in range(0,1):
        if  mae2["Best"][0]=="RF":
            bm2_dict[sku]=RF_final(X2,y);
                    
        elif mae2["Best"][0]=="NN":
            bm2_dict[sku]=MLP_final(X2,y);
                                
        elif mae2["Best"][0]=="GB":
            bm2_dict[sku]=GB_final(X2,y); 
                        
        elif mae2["Best"][0]=="XG":
            bm2_dict[sku]=XG_final(X2,y);    

In [56]:
tier_pred=pd.concat([bm_dict[0][0],bm2_dict[0][0],y],axis=1)
tier_pred.columns = ['DC_Phase1', 'DC_Phase2',"Sales"]
# tier_pred.to_csv('C:\\Users\\lengada1\\NCSU\\DC_tier_predictions.csv')
tier_pred.to_csv('./DC_tier_predictions.csv')
tier_pred.round()

,DC_Phase1,DC_Phase2,Sales
0,72391.0,72426.0,72552
1,64637.0,64343.0,64297
2,63663.0,63823.0,64112
3,65037.0,64988.0,65006
4,43223.0,43305.0,43300
5,-5.0,4.0,0
6,50906.0,50998.0,50984
7,48195.0,48186.0,48246
8,45988.0,45686.0,45400
9,47282.0,46988.0,46978


In [57]:
# END NOTEBOOK
bm_dict[0][1]

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=8, max_features=14,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [58]:
def save_model(model, filename):
    """
    Saves the model as a file. 
    - Keras models: saved as .json and .h5 files
    - scikit-learn models: saved as .pickle files
    """
    
    if type(model) is KerasRegressor:
        
        model_json = model.model.to_json()
        with open('{}.json'.format(filename), "w") as json_file:
            json_file.write(model_json)
        model.model.save_weights('{}.h5'.format(filename))        
    else:
        pickle_out = open('{}.pickle'.format(filename), "wb")
        pickle.dump(model, pickle_out)
        pickle_out.close()


# Save model to disk
filename="NCSU_DC"
model = bm_dict[0][1]
save_model(model, filename)


In [59]:
# Load model from disk
def load_pickled_model(path_to_model):
    loaded_model = pickle.load(open(path_to_model, 'rb'))
    
    return loaded_model

def load_keras_model(path_to_json, path_to_weights_h5):
    
    # load json and create model
    json_file = open(path_to_json, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    # load weights into new model
    loaded_model.load_weights(path_to_weights_h5)
    
    # wrap Sequential obj as a KerasRegressor to allow scikit-learn compatibility
    wrapped_loaded_model = KerasRegressor(build_fn=create_MLP, epochs=400, batch_size=20, verbose=0)
    
    return wrapped_loaded_model

def load_model(filename):
    """
    Loads a given file as a model
    - .pickle files: loaded as scikit-learn models
    - .json and .h5 files: loaded together to create Keras models
    """
    
    loaded_model = None
    
    try:
        loaded_model = load_pickled_model("{}.pickle".format(filename))
        
    except FileNotFoundError:
        loaded_model = load_keras_model("{}.json".format(filename), "{}.h5".format(filename))
        
    return loaded_model

loaded_model = load_model("NCSU_DC")

In [60]:
# Evaluate loaded model on test data
scores=[]
model_family=[]
    
# Identify the model family
if isinstance(loaded_model, RandomForestRegressor):
    model_family.append('RF')
    
elif isinstance(loaded_model, xgb.XGBRegressor):
    model_family.append('XG')   
    
elif isinstance(loaded_model, ensemble.GradientBoostingRegressor):
    model_family.append('GB')    
    
else:
    model_family.append('MLP')

# Obtain the cross validation score
score = -cross_val_score(loaded_model,X,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)
scores.append(score)        

loaded_model_scores = pd.DataFrame({
    'CV Score':scores,
    'Family':model_family
})

loaded_model_scores

,CV Score,Family
0,3106.0,GB
